Python: 3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]

|모듈|버젼|
|----|----|
|pandas|0.25.1|
|numpy|1.18.5|
|sklearn|0.21.3|
|scipy|1.5.2|
|mlxtend|0.15.0.0|
|statsmodels|0.11.1|
|imblearn|0.5.0|
|xgboost|0.80|

In [2]:
import pandas as pd
import numpy as np
import sklearn
import scipy 
import mlxtend
import statsmodels
import imblearn
import xgboost as xgb
import sys

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, imblearn, xgb]:
    print(i.__name__,  i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
imblearn 0.5.0
xgboost 0.80


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

# 대문항 세트1

포켓몬스터 게임(휴대용 캡슐로 몬스터를 포획하여 주머니 속에 간편하게 데리고 다니며 

전투를 수행하며 몬스터를 키우는 게임) 마니아인 당신은 포켓몬(게임 속 몬스터)의 속성과 전투 이력을 분석하여 게임 전략을 세우고자 합니다. 

포켓몬의 특성에 따른 승률을 분석하기 위하여 두 종류의 데이터 셋을 수집하였으며, 데이터 특성은 아래와 같습니다.

□ 포켓몬 정보 데이터

**prof2105_1_1.csv** (구분자: Comma(,), 800 Rows, 12 Columns, UTF-8 인코딩)

|컬럼|정의|타입|
|----|----|----|
|id|포켓몬ID|Double|
|name|포켓몬이름|String|
|type|속성|String|
|total|능력치 총합|Double|
|hp|체력 능력치|Double|
|attack|공격 능력치|Double|
|defense|방어 능력치|Double|
|spc_attack|특수 공격 능력치|Double|
|spc_defense|특수 방어 능력치|Double|
|speed|속도 능력치|Double|
|generation|세대|Double|
|legendary|전설의 포켓몬 여부(True/False)|String|

포켓몬 정보 데이터는 포켓몬 이름 별로 한 행씩 존재합니다.

단, 돌연변이 포켓몬이 발생할 경우가 있으며, 

이 경우 포켓몬ID는 동일하지만 이름은 다른 포켓몬이 존재할 수 있습니다. 

즉, 포켓몬ID와 포켓몬 이름은 1:1 혹은 1:N의 관계입니다.

각 포켓몬은 ‘Grass’, ‘Fire’ 등의 속성을 가지고 있으며, 

일부 포켓몬은 두 가지 속성을 보유하고 있습니다. 

이 경우 ‘Grass/Poison’과 같이 ‘/’ 문자를 구분자로 두 가지 속성 정보가 결합되어 있습니다.

In [6]:
df_mon_org = pd.read_csv('data/prof2105_1_1.csv')
df_mon_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
id             800 non-null float64
name           800 non-null object
type           800 non-null object
total          800 non-null float64
hp             800 non-null float64
attack         800 non-null float64
defense        800 non-null float64
spc_attack     800 non-null float64
spc_defense    800 non-null float64
speed          800 non-null float64
generation     800 non-null float64
legendary      800 non-null bool
dtypes: bool(1), float64(9), object(2)
memory usage: 69.7+ KB


In [7]:
df_mon_org.head()

,id,name,type,total,hp,attack,defense,spc_attack,spc_defense,speed,generation,legendary
0,1.0,Bulbasaur,Grass/Poison,318.0,45.0,49.0,49.0,65.0,65.0,45.0,1.0,False
1,2.0,Ivysaur,Grass/Poison,405.0,60.0,62.0,63.0,80.0,80.0,60.0,1.0,False
2,3.0,Venusaur,Grass/Poison,525.0,80.0,82.0,83.0,100.0,100.0,80.0,1.0,False
3,3.0,VenusaurMega Venusaur,Grass/Poison,625.0,80.0,100.0,123.0,122.0,120.0,80.0,1.0,False
4,4.0,Charmander,Fire,309.0,39.0,52.0,43.0,60.0,50.0,65.0,1.0,False


□ 전투 정보 데이터

**prof2105_1_2.csv** (구분자: Comma(,), 800 Rows, 12 Columns, UTF-8 인코딩)

|컬럼|정의|타입|
|----|----|----|
|id1|선공격 포켓몬ID|Double|
|id2|후공격 포켓몬ID|Double|
|winner|승리한 포켓몬ID(id1, id2 중 승리한 포켓몬ID )|Double|

전투 정보 데이터는 각 전투에 참여한 두 포켓몬ID와 두 포켓몬ID 중 전투에서 승리한 포켓몬ID로 이루어져 있으며, 

아직 전투를 실행하지 않은 경우, 승리 포켓몬 ID 정보는 비어있습니다(null).

‘서로 비기는 경우’와 ‘전투에 참여한 두 포켓몬ID가 서로 동일한 경우’는 존재하지 않습니다.

두 포켓몬ID 간에 여러 번 전투가 이루어 질 수 있으며, 승리 결과는 매번 다를 수 있습니다.


In [8]:
df_battle_org = pd.read_csv('data/prof2105_1_2.csv')
df_battle_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
id1       60000 non-null float64
id2       60000 non-null float64
winner    50000 non-null float64
dtypes: float64(3)
memory usage: 1.4 MB


In [9]:
df_battle_org.head()

,id1,id2,winner
0,266.0,298.0,298.0
1,702.0,701.0,701.0
2,191.0,668.0,668.0
3,237.0,683.0,683.0
4,151.0,231.0,151.0


## 공통 전처리

**(prof2105_1_1.csv, prof2105_1_2.csv 를 이용하여)** 분석에 앞서 다음의 전처리를 순차적으로 수행하시오.

**Step A-1**: 

prof2105_1_1.csv 데이터 셋의 속성(type) 컬럼에서 “/” 앞의 첫 속성 정보만 가지도록 추출합니다.

단, 변환 시 컬럼명은 type으로 유지하시오. (변환 예시: “Grass/Poison” → “Grass”)


In [12]:
# 방법#1
df_mon = df_mon_org.copy()
df_mon['type'] = df_mon['type'].apply(lambda x: x.split('/')[0])

In [19]:
# 방법#2
df_mon = df_mon_org.copy()
df_mon['type'] = df_mon['type'].str.extract('(?P<type1>[a-zA-Z]+)')

**Step A-2**

포켓몬ID(id)별로 캐릭터가 여러 개 존재하기 때문에, 포켓몬ID(id) 별 고유한 정보를 가지도록 데이터를 가공하고자 합니다. 

Step A-1에서 처리한 데이터를 ‘포켓몬ID(id), 속성(type), 전설의 포켓몬 여부(legendary)’별로 아래의 6개 변수의 평균을 집계합니다. 

단, 집계된 평균 값은 집계 전 컬럼명과 동일하게 유지하시오. (처리 결과: 721 Rows)

- 평균 계산 변수(총 6개): 체력 능력치(hp), 공격 능력치(attack), 방어 능력치(defense), 특수 공격 능력치(spc_attack), 특수 방어 능력치(spc_defense), 속도 능력치(speed)


In [26]:
mean_cols = ['hp', 'attack', 'defense', 'spc_attack', 'spc_defense', 'speed']
df_mon_stat = df_mon.groupby(['id', 'type', 'legendary'])[mean_cols].mean().reset_index(level=[1, 2])
df_mon_stat.head()

,type,legendary,hp,attack,defense,spc_attack,spc_defense,speed
id,,,,,,,,
1.0,Grass,False,45.0,49.0,49.0,65.0,65.0,45.0
2.0,Grass,False,60.0,62.0,63.0,80.0,80.0,60.0
3.0,Grass,False,80.0,91.0,103.0,111.0,110.0,80.0
4.0,Fire,False,39.0,52.0,43.0,60.0,50.0,65.0
5.0,Fire,False,58.0,64.0,58.0,80.0,65.0,80.0


**Step A-3**
    
prof2105_1_2.csv 데이터 셋에서 승리 포켓몬ID(winner)가 null이 아닌 데이터를 추출합니다. 

이후 784개 포켓몬ID별 승률을 계산하여 승률 (변수명: win_rate, 범위: 0~1) 컬럼을 생성합니다. 

- 승률은 해당 포켓몬이 참가한 전체 전투 중 이긴 전투(win_yn=1)의 비율을 의미합니다.
- 승률 계산 시 캐릭터가 선공격 포켓몬(id1)인 경우와 후공격 포켓몬(id2)인 경우를 모두 고려하여야 합니다. 
- 포켓몬ID별 승률 결과는 총 784 Rows로 구성됩니다.


In [36]:
# 방법1
s_win_rate = pd.concat([
    df_battle_org.loc[df_battle_org['winner'].notna(), ['id1', 'winner']].assign(
        win_yn = lambda x: x['id1'] == x['winner']
    )[['id1', 'win_yn']].rename(columns={'id1': 'id'}),
    df_battle_org.loc[df_battle_org['winner'].notna(), ['id2', 'winner']].assign(
        win_yn = lambda x: x['id2'] == x['winner']
    )[['id2', 'win_yn']].rename(columns={'id2': 'id'})
]).groupby('id')['win_yn'].mean().rename('win_rate')
s_win_rate

id
1.0      0.278195
2.0      0.380165
3.0      0.674242
4.0      0.560000
5.0      0.491071
           ...   
796.0    0.371429
797.0    0.885496
798.0    0.504202
799.0    0.618056
800.0    0.619835
Name: win_rate, Length: 784, dtype: float64

In [53]:
# 방법 2
s_win_rate = df_battle_org.loc[df_battle_org['winner'].notna()]\
    .melt(id_vars=['winner']).assign(
        win_yn = lambda x: x['winner'] == x['value']
    ).groupby('value')['win_yn'].mean().rename('win_rate')
s_win_rate

value
1.0      0.278195
2.0      0.380165
3.0      0.674242
4.0      0.560000
5.0      0.491071
           ...   
796.0    0.371429
797.0    0.885496
798.0    0.504202
799.0    0.618056
800.0    0.619835
Name: win_rate, Length: 784, dtype: float64

**Step A-4**

Step A-2 처리 결과 데이터와 Step A-3 처리 결과 데이터를 포켓몬ID(id) 컬럼을 Join Key로 하여 행 결합(Inner join)합니다. 

이후 포켓몬ID(id) 컬럼을 기준으로 오름차순 정렬합니다. 

(데이터셋명: **attr**, 706 Rows)

상기 전처리를 완료한 데이터 셋을 ‘attr’ (706 Rows)로 정의하시오.

(이후 문항에서의 지칭을 위하여 데이터셋 이름을 정의한 것으로, 제출 소스코드에서 반드시 위의 이름을 따를 필요는 없다.)


In [56]:
df_attr = df_mon_stat.join(s_win_rate, how='inner')
df_attr

,type,legendary,hp,attack,defense,spc_attack,spc_defense,speed,win_rate
1.0,Grass,False,45.0,49.0,49.0,65.0,65.0,45.0,0.278195
2.0,Grass,False,60.0,62.0,63.0,80.0,80.0,60.0,0.380165
3.0,Grass,False,80.0,91.0,103.0,111.0,110.0,80.0,0.674242
4.0,Fire,False,39.0,52.0,43.0,60.0,50.0,65.0,0.560000
5.0,Fire,False,58.0,64.0,58.0,80.0,65.0,80.0,0.491071
...,...,...,...,...,...,...,...,...,...
717.0,Dark,True,126.0,131.0,95.0,131.0,98.0,99.0,0.959350
718.0,Dragon,True,108.0,100.0,121.0,81.0,95.0,95.0,0.820312
719.0,Rock,True,50.0,130.0,130.0,130.0,130.0,80.0,0.175676
720.0,Psychic,True,80.0,135.0,60.0,160.0,130.0,75.0,0.403361


# 쉬는 시간: ~ 10:18

## 문제 1번 10점

(attr을 이용하여) 전설의 포켓몬은 일반 포켓몬보다 강력하며, 희귀한 몬스터입니다. 

전설의 포켓몬 여부(legendary)에 따라 승률(win_rate)의 평균에 통계적으로 유의미한 차이가 존재하는지 검정하고자 합니다. 

아래 단계에 따라 적절한 검정을 수행하고, 수행 결과의 유의 확률(p-value)을 기술하시오.

**Step 1-1** 

전설의 포켓몬 여부(legendary)별 승률(win_rate)의 등분산성을 확인하기 위하여 Bartlett’s Test를 수행하고, 

유의수준 0.05하에 등분산 여부를 검정하시오.

- 대립가설(H1): 전설의 포켓몬 여부(legendary)에 따라 승률(win_rate) 분산이 다르다.

**Python 함수가이드**
```
from scipy.stats import bartlett
문제에서 지시한 조건 외 Default 값 사용
```

**Step 1-2**

전설의 포켓몬 여부(legendary)별 승률(win_rate) 평균이 통계적으로 차이가 있는지 검정하기 위하여 적절한 모수적(Parametric) 검정을 수행합니다. 

검정 수행 시, Step 1-1에서 확인한 등분산 여부를 검정 옵션에 반영하시오.

- 대립가설(H1): 전설의 포켓몬 여부(legendary)에 따라 승률(win_rate) 평균에 차이가 있다.
    
**Python 함수가이드**
```
from scipy.stats import ttest_ind 
문제에서 지시한 조건 외 Default 값 사용
```
유의확률은 소수점 다섯째 자리에서 반올림하여 소수점 넷째 자리까지 기술하시오.

(답안예시) 0.1234

**Step1-α1**

포켓몬의 type에 따라 win_rate의 등분산인지 아닌지를 검정하고자 합니다. 

Step 1-1에서 사용했던 bartlett 검정을 하고, pvalue를 구합니다.

**Step1-α2**

포켓몬의 type에 따라 win_rate의 평균이 같은지 아닌지를 모수적 방법으로 검정하세요

**Python 함수가이드**
```
from scipy import stats
에서 적절한 기능을 선택하여 사용합니다.
문제에서 지시한 조건 외 Default 값 사용
```

**Step1-α3**

포켓못의 type과 전설여부에 따라 win_rate의 평균이 같은지 아닌지와 두 범주의 교호(상호)작용 파악하기 위해 이원산 분산분석(two-way ANOVA) 검정을 수행하고 교호항의 검정통계량을 구하세요

포켓못 type 중에서 legendary 포켓몬은 5개 이상 포함한 type에 대해서 진행합니다.

**Python 함수가이드**
```
from statsmodels.formula import api

from statsmodels.api import stats

문제에서 지시한 조건 외 Default 값 사용
```

## 문제 2번 10점

**(attr를 이용하여)** 포켓몬의 6개 능력치와 승률을 이용하여 전설의 포켓몬 여부를 예측하는 모형을 만들고자 합니다. 

이를 위하여 아래의 과정을 수행하고 질문에 답하시오.

**Step 2-1.**

아래 가이드에 따라 Logistic Regression 모델을 학습하고, 

학습한 모델을 동일한 데이터 셋에 적용하여 전설의 포켓몬 여부(legendary)를 예측하시오.

- 종속변수: 전설의 포켓몬 여부(legendary)

- 독립변수(총 7개, 모형 학습 시 변수 입력 순서 유지): 체력 능력치(hp), 공격 능력치(attack), 방어 능력치(defense), 특수 공격 능력치(spc_attack), 특수 방어 능력치 (spc_defense), 속도 능력치(speed), 승률(win_rate)

- 상수항(Intercept)을 포함하여 학습


**Python 함수가이드**
```
from sklearn.linear_model import LogisticRegression
	solver='lbfgs'
	random_state=1234
	C=100000
문제에서 지시한 조건 외 Default 값 사용
```

**Step 2-2.**

정확도(Accuracy)를 다음과 같이 정의할 때,

$정확도(Accuracy)=\frac{예측값과 실제값이 일치하는 데이터의 건수}{전체 데이터의 건수}×100$

계산된 정확도(단위: %, 범위: 0~100)를 소수점 둘째 자리에서 반올림하여 소수점 첫째 자리까지 기술하시오.

(답안예시) 12.3

## 문제 3번 15점

**(attr, prof2105_1_2.csv 을 이용하여)**

전투에 참가한 두 포켓몬의 능력치를 입력변수로 하여 각 전투의 승자를 예측하는 모델을 개발하고자 합니다.

이를 위해 아래의 과정을 수행하고 질문에 답하시오.

**Step 3-1** 

prof2105_1_2.csv 데이터에 승리 포켓몬ID(winner)가 선공격 포켓몬ID(id1)와 동일할 경우 1, 

후공격 포켓몬ID(id2)와 동일할 경우 0, null일 경우 null의 값을 갖는 선공격 승리 여부(win_1) 컬럼을 추가하시오. 

(데이터셋명: **battle**, 60,000 Rows)


**Step 3-2**

전투 정보에 전투에 참가한 두 포켓몬ID의 속성을 다음과 같이 결합합니다. 

battle과 attr을 결합(Inner join, Join key: battle의 id1 = attr의 id)하여 battle_1이라 정의하고, 

battle_1과 attr을 또다시 결합(Inner join, Join key: battle_1의 id2=attr의 id)하여 battle_2라 정의합니다.

- attr에서 가져올 컬럼(총 7개): 속성(type), 체력 능력치(hp), 공격 능력치(attack), 방어 능력치(defense), 특수 공격 능력치(spc_attack), 특수 방어 능력치(spc_defense), 속도 능력치(speed)

- 단, attr 테이블을 두 번 결합하므로 컬럼명의 중복이 발생합니다. 이를 방지하기 위하여 선공격 포켓몬ID(id1)에 대한 컬럼인 경우, 
    대상 컬럼명 뒤에 ‘_1’를 붙이고, 후공격 포켓몬ID(id2)에 대한 컬럼인 경우에는 컬럼명 뒤에 ‘_2’를 붙이시오.

- 결합 후 선공격 포켓몬 속성(type_1)과 후공격 포켓몬 속성(type_2)이 모두 null이 아닌 행만 남기시오. (Inner Join 결과: 48,364 Rows)


**Step 3-3**

battle_2에서 선공격 포켓몬의 속성(type_1)과 후공격 포켓몬의 속성(type_2)이 같은 데이터는 제거하시오. 

이후 선공격 포켓몬ID(id1), 후공격 포켓몬ID(id2) 두 컬럼을 기준으로 오름차순 정렬합니다. 

(제거 결과: 44,618 Rows)

Step3-3 처리 결과 예시]

|id1|id2|winner|win_1|type_1|hp_1|…|speed_2|
|---|---|------|-----|------|----|--|-------|
|1|6|6|0|Grass|45| |100|
|1|26|26|0|Grass|45| |110|


**Step 3-4**

Step3-3 결과 데이터를 아래 조건에 따라 학습/테스트 데이터를 battle_train / battle_test로 분할하시오.

- battle_train: 승리 포켓몬ID(winner)가 null이 아닌 데이터 (37,480 Rows)

- battle_test: 승리 포켓몬ID(winner)가 null인 데이터 (7,138 Rows)


**Step 3-5**

battle_train 데이터를 이용하여 아래 가이드에 따라 Random Forest 분류 모델을 학습한 후,

학습한 모델을 동일한 데이터 셋(battle_train)에 적용하여 선공격 포켓몬 승리 여부(win_1)를 예측합니다.

- 종속변수: 선공격 승리 여부(win_1)
- 독립변수(총 12개, 입력 순서 유지): hp_1, attack_1, defense_1, spc_attack_1, spc_defense_1, speed_1, hp_2, attack_2, defense_2, spc_attack_2, spc_defense_2, speed_2

**Python 함수가이드**
```
from sklearn.ensemble import RandomForestClassifier
random_state = 1234, n_estimators=10 
문제에서 지시한 조건 외 Default 값 사용
```

예측 결과에서 실제 '선공격 포켓몬 승리 여부(win_1)'가 1인 전투 중, 

예측 결과도 1인 전투의 비율(%, 범위: 0~100)을 계산하시오. 

계산된 값은 소수점 둘째 자리에서 반올림하여 소수점 첫째 자리까지 기술하시오.

(답안예시) 12.3

## 문제 4번 15점

(battle_train, battle_test를 이용하여) 이번에는 전투에 참가한 두 포켓몬의 능력치를 입력변수로 하여 

속성별 예상 승률을 계산하고자 합니다. 이를 위해 아래의 과정을 수행하고 질문에 답하시오.

**Step 4-1**

battle_train을 이용하여 Naïve Bayes 모형을 학습하시오.

- 종속변수: 선공격 승리 여부(win_1)
- 독립변수(총 12개, 입력 순서 유지): hp_1, attack_1, defense_1, spc_attack_1, spc_defense_1, speed_1, hp_2, attack_2, defense_2, spc_attack_2, spc_defense_2, speed_2 

**Python 함수가이드**
```
from sklearn.naive_bayes import MultinomialNB
문제에서 지시한 조건 외 Default 값 사용
```

**Step 4-2**

battle_test에 훈련된 모형을 적용하여 각 전투의 승리 캐릭터를 예측 후, 

18개의 속성(type)별 승률(단위:%, 범위: 0~100)을 계산하시오. 

- 승률은 해당 포켓몬이 참가한 모든 전투 중 이긴 전투(win_yn=1)의 비율을 의미합니다.
- 승률 계산 시 해당 포켓몬이 선공격 포켓몬ID(id1)인 경우와 후공격 포켓몬ID(id2)인 경우를 모두 고려하여야 합니다.
- Test set의 각 전투 데이터는 선공격 포켓몬의 속성(type_1)과 후공격 포켓몬의 속성(type_2)이 서로 다르게 구성되어 있습니다.

속성(type)별 승률 계산 결과, 승률이 가장 높은 속성(type)의 승률을 소수점 둘째 자리에서 반올림하여 소수점 첫째 자리까지 기술하시오.

(답안예시) 12.3
